<a href="https://colab.research.google.com/github/pea-sys/Til/blob/master/PyTorch%E6%9C%AC7_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.4 word2vec、fastTextを用いた日本語単語のベクトル表現の実装
本ファイルでは、日本語の単語をword2vecもしくはfastTextを使用してベクトル化する手法を解説します。
※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.4 学習目標
学習済みの日本語word2vecモデルで単語をベクトル表現に変換する実装ができるようになる
学習済みの日本語fastText モデルで単語をベクトル表現に変換する実装ができるようになる

[写経元](https://github.com/YutaroOgawa/pytorch_advanced/blob/master/7_nlp_sentiment_transformer/7-4_vectorize.ipynb)

In [0]:
import os
import urllib.request
import zipfile
import tarfile

In [0]:
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# word2vec学習済みモデルをダウンロード

In [0]:
# word2vecの日本語学習済みモデル（東北大学 乾・岡崎研究室）をダウンロード。時間が15分ほどかかります

url = "http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2"
save_path = "./data/20170201.tar.bz2"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# './data/20170201.tar.bz2'の解凍　5分ほどかかります

# tarファイルを読み込み
tar = tarfile.open('./data/20170201.tar.bz2', 'r|bz2')
tar.extractall('./data/')  # 解凍
tar.close()  # ファイルをクローズ

# フォルダ「data」内にフォルダ「entity_vector」というものができ、その中に「entity_vector.model.bin」というファイルができています。

# fastTextの英語学習済みモデルをダウンロード

In [0]:
# fastTextの公式の英語学習済みモデル（650MB）をダウンロード。時間が5分ほどかかります
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
save_path = "./data/wiki-news-300d-1M.vec.zip"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# フォルダ「data」内の「/wiki-news-300d-1M.vec.zip」を解凍する

zip = zipfile.ZipFile("./data/wiki-news-300d-1M.vec.zip")
zip.extractall("./data/")  # ZIPを解凍
zip.close()  # ZIPファイルをクローズ

# フォルダ「data」内にフォルダ「wiki-news-300d-1M.vec」というものができます。

# IMDbデータセットをダウンロード

In [0]:
# IMDbデータセットをダウンロード。30秒ほどでダウンロードできます

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = "./data/aclImdb_v1.tar.gz"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# './data/aclImdb_v1.tar.gz'の解凍　1分ほどかかります

# tarファイルを読み込み
tar = tarfile.open('./data/aclImdb_v1.tar.gz')
tar.extractall('./data/')  # 解凍
tar.close()  # ファイルをクローズ

# フォルダ「data」内にフォルダ「aclImdb」というものができます。

In [9]:
!pip install janome

     |████████████████████████████████| 21.5MB 198kB/s 


In [10]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)

機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [11]:
# 単語分割する関数を定義


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


text = '機械学習が好きです。'
print(tokenizer_janome(text))

['機械', '学習', 'が', '好き', 'です', '。']


In [12]:
!sudo apt install file
#MeCabのインストール
!sudo apt install mecab
!sudo apt install libmecab-dev
!sudo apt install mecab-ipadic-utf8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 25 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,294 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.3 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.3 [68.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.3 [22.1 kB]
Fetched 275 kB in 1s (382 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program i

In [13]:
!git clone https://github.com/neologd/mecab-ipadic-neologd.git
%cd mecab-ipadic-neologd
!sudo bin/install-mecab-ipadic-neologd

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 9143, done.
remote: Total 9143 (delta 0), reused 0 (delta 0), pack-reused 9143
Receiving objects: 100% (9143/9143), 413.05 MiB | 44.23 MiB/s, done.
Resolving deltas: 100% (5270/5270), done.
/content/mecab-ipadic-neologd
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-

In [14]:
!pip install mecab-python3

     |████████████████████████████████| 17.1MB 230kB/s 


In [70]:
import google.colab
import googleapiclient.discovery
import googleapiclient.http
google.colab.auth.authenticate_user()
drive_service = googleapiclient.discovery.build('drive', 'v3')

def gdrive_dl(filename):
  upload_filename = filename

  file_list = drive_service.files().list(q="name='" + upload_filename + "'").execute().get('files')

  # ファイル ID を取得します。
  file_id = None
  for file in file_list:
    if file.get('name') == upload_filename:
      file_id = file.get('id')
      break

  if file_id is None:
    # ファイル ID を取得できなかった場合はエラーメッセージを出力します。
    print(upload_filename + ' が見つかりません.')
  else:
    # colab 環境へファイルをアップロードします。
    with open(upload_filename, 'wb') as f:
      request = drive_service.files().get_media(fileId=file_id)
      media = googleapiclient.http.MediaIoBaseDownload(f, request)

      done = False
      while not done:
        progress_status, done = media.next_chunk()
        print(100*progress_status.progress(), end="")
        print("%完了")

    print('GoogleドライブからColab環境へのファイル取り込みが完了しました.')

gdrive_dl('vector_neologd.zip')
gdrive_dl('model.vec')


30.20688780734656%完了
60.41377561469312%完了
90.62066342203968%完了
100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.
18.921911328992643%完了
37.843822657985285%完了
56.76573398697792%完了
75.68764531597057%完了
94.6095566449632%完了
100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了しました.


In [0]:

# フォルダ「data」内の「vector_neologd.zip」を解凍する

zip = zipfile.ZipFile("vector_neologd.zip")
zip.extractall("./data/vector_neologd/")  # ZIPを解凍
zip.close()  # ZIPファイルをクローズ

In [0]:
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [0]:
# 単語分割にはMecab＋NEologdを使用
import MeCab

m_t = MeCab.Tagger("-Owakati -d {0}".format(path))#MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret



# 前処理として正規化をする関数を定義
import re

def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Mecabの単語分割

    return ret

In [46]:
!git clone https://github.com/YutaroOgawa/pytorch_advanced.git
%cd pytorch_advanced/7_nlp_sentiment_transformer

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 441, done.
remote: Total 441 (delta 0), reused 0 (delta 0), pack-reused 441
Receiving objects: 100% (441/441), 14.62 MiB | 26.41 MiB/s, done.
Resolving deltas: 100% (231/231), done.


In [0]:

import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


# フォルダ「data」から各tsvファイルを読み込みます
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 2. 単語のベクトル化
# 2.1 word2vec
単語をベクトル表現に変換します。

TorchTextには日本語の学習済みデータがないわけではないですが、精度が微妙なので

東北大学 乾・岡崎研究室で公開されているWord2Vecの学習済みのベクトルを使用します。

In [20]:
!pip install gensim

In [32]:

# そのままではtorchtextで読み込めないので、gensimライブラリを使用して、
# Word2Vecのformatで保存し直します

from gensim.models import KeyedVectors


# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    '/content/data/entity_vector/entity_vector.model.bin', binary=True)#./data/entity_vector/entity_vector.model.bin

# 保存（時間がかかります、10分弱）
model.wv.save_word2vec_format('/content/data/japanese_word2vec_vectors.vec')#./data/japanese_word2vec_vectors.vec

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [35]:
# torchtextで単語ベクトルとして読み込みます
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='/content/data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))

100%|█████████▉| 1014375/1015474 [01:30<00:00, 11932.35it/s]

1単語を表現する次元数： 200
単語数： 1015474


In [55]:

# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 49個の単語が200次元のベクトルで表現されている
TEXT.vocab.vectors

torch.Size([49, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-2.8353,  2.5609, -0.5348,  ...,  0.4602,  1.4669, -2.1255],
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.0448, -0.1304,  0.0329,  ...,  0.0825, -0.1386,  0.0417]])

In [56]:
# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'0': 18,
             '<pad>': 1,
             '<unk>': 0,
             '、': 7,
             '。': 3,
             'い': 19,
             'いる': 11,
             'か': 12,
             'から': 20,
             'が': 8,
             'し': 9,
             'する': 21,
             'その': 22,
             'た': 23,
             'て': 13,
             'で': 24,
             'です': 25,
             'と': 2,
             'な': 4,
             'に': 26,
             'に対して': 27,
             'の': 5,
             'は': 28,
             'まし': 29,
             'ます': 14,
             'を': 10,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '分類': 15,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
             '女王': 39,
             '好き': 40,
             '姫': 41,
             '文章': 6,
             '本章': 16,
      

In [65]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))

女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


# 2.2 fastText

word2vecより進歩したベクトル化手法であるfastTextによる単語のベクトル表現を使用します。

日本語の学習モデルを以下の記事にて公開してくださっているので、使用させていただきます。

In [0]:
# Qiita：いますぐ使える単語埋め込みベクトルのリスト
# https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

# Download Word Vectors
# https://drive.google.com/open?id=0ByFQ96A4DgSPNFdleG1GaHcxQzA

In [72]:
# torchtextで単語ベクトルとして読み込みます
# word2vecとは異なり、すぐに読み込めます

from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='model.vec')#./data/vector_neologd/

                                    
# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：", len(japanese_fasttext_vectors.itos))


  0%|          | 0/211673 [00:00<?, ?it/s]Skipping token b'211673' with 1-dimensional vector [b'300']; likely a header

100%|█████████▉| 211158/211673 [00:24<00:00, 8318.56it/s]

1単語を表現する次元数： 300
単語数： 211673


In [73]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

torch.Size([49, 300])


defaultdict(<function torchtext.vocab._default_unk_index>,
            {'0': 18,
             '<pad>': 1,
             '<unk>': 0,
             '、': 7,
             '。': 3,
             'い': 19,
             'いる': 11,
             'か': 12,
             'から': 20,
             'が': 8,
             'し': 9,
             'する': 21,
             'その': 22,
             'た': 23,
             'て': 13,
             'で': 24,
             'です': 25,
             'と': 2,
             'な': 4,
             'に': 26,
             'に対して': 27,
             'の': 5,
             'は': 28,
             'まし': 29,
             'ます': 14,
             'を': 10,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '分類': 15,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
             '女王': 39,
             '好き': 40,
             '姫': 41,
             '文章': 6,
             '本章': 16,
      

In [74]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))

女王 tensor(0.2987)
王 tensor(0.3364)
王子 tensor(0.4068)
機械学習 tensor(0.)
